`Q1 Running Elastic`

In [1]:
!curl localhost:9200

{
  "name" : "a5e21b49b3be",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "fRejSsSiQ_6NUcJdGvVDpg",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


**Getting the Data**

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

`Q2 Indexing the Data`

In [3]:
from elasticsearch import Elasticsearch

In [4]:
es_client = Elasticsearch('http://localhost:9200/')

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-new"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-new'})

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

`Q3 Searching`

In [10]:
query = "How do execute a command on a Kubernetes pod?"

In [11]:
def elastic_search(query,course="data-engineering-zoomcamp",score=False):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                       "boost":4,
                        "fields": ["question", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": f"{course}"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        if score:
            result_docs.append(hit['_score'])
    
    return result_docs

In [12]:
elastic_search(query,score=True)

[{'text': "psql is a command line tool that is installed alongside PostgreSQL DB, but since we've always been running PostgreSQL in a container, you've only got `pgcli`, which lacks the feature to run a sql script into the DB. Besides, having a command line for each database flavor you'll have to deal with as a Data Professional is far from ideal.\nSo, instead, you can use usql. Check the docs for details on how to install for your OS. On macOS, it supports `homebrew`, and on Windows, it supports scoop.\nSo, to run the taxi_trips.sql script with usql:",
  'section': 'Workshop 2 - RisingWave',
  'question': 'psql - command not found: psql (alternative install)',
  'course': 'data-engineering-zoomcamp'},
 35.6804,
 {'text': 'If you are running Jupyter Notebook on a fresh new Codespace or in local machine with a new Virtual Environment, you will need this package to run the starter Jupyter Notebook offered by the teacher. Execute this:\npip install jupyter',
  'section': 'Workshop 1 - dlt

`Q4 Filtering`

In [13]:
elastic_search(query,course="machine-learning-zoomcamp")

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Problem: CPU metrics Shows Unknown\nNAME         REFERENCE           TARGETS         MINPODS   MAXPODS   REPLICAS   AGE\ncredit-hpa   Deployment/credit   <unknown>/20%   1         3         1          18s\nFailedGetResourceMetric       2m15s (x169 over 44m)  horizontal-pod-autoscaler  failed to get cpu utilization: unable to get metrics for resource cpu: unable to fetch metrics from resource metrics API:\nSolution:\n-> Delete HPA (kubectl delete hpa credit-hpa)\n-> kubectl apply -f https://raw.githubu

`Q5 Building a Prompt`

In [14]:
context=""
for i in elastic_search(query,course="machine-learning-zoomcamp"):
    context=context+f"question: {i['question']}\nanswer: {i['text']}\n\n" 

In [15]:
len(context)

3060

In [16]:
def build_prompt(query, context):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

`Q6 Tokens`

In [18]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 19.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
import tiktoken

prompt = build_prompt(query, context)

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
print(len(tokens))

739
